In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import datetime
import json
import pandas as pd
import numpy as np

In [2]:
#set up engine
engine = create_engine("sqlite:///../Data/Poker_data.db")

In [3]:
#Verify that the database is connected and get table names for reference
inspector = inspect(engine)
inspector.get_table_names()

['dbo_hand_roster', 'dbo_hands', 'dbo_players']

In [4]:
#Verify column names for queries and future use
columns = inspector.get_columns('dbo_hands')
column_names = []
for c in columns:
    column_names.append(c['name'])
    print(c['name'], c["type"])

hand_id BIGINT
table_num INTEGER
table_hand_num INTEGER
num_players INTEGER
flop_pot TEXT
turn_pot TEXT
river_pot TEXT
winnings_pot TEXT
table_1 TEXT
table_2 TEXT
table_3 TEXT
table_4 TEXT
table_5 TEXT


In [5]:
#run queries for data
hands_df = pd.read_sql('SELECT hand.num_players, hand.flop_pot, hand.turn_pot, hand.river_pot, player.chip_count, player.chair_num, player.player_bet, player.player_winnings, hand.table_1, hand.table_2, hand.table_3, hand.table_4, hand.table_5 FROM dbo_hands hand JOIN dbo_players player ON hand.hand_id = player.hand_id ORDER BY hand.hand_id DESC', engine)
hands_df.head(100)

,num_players,flop_pot,turn_pot,river_pot,chip_count,chair_num,player_bet,player_winnings,table_1,table_2,table_3,table_4,table_5
0,2,0/0,0/0,0/0,1995,1,10,20,None,None,None,None,None
1,2,0/0,0/0,0/0,50,2,10,0,None,None,None,None,None
2,2,2/20,2/20,0/0,1045,2,970,1920,8c,Td,2d,Qs,None
3,2,2/20,2/20,0/0,1000,1,950,0,8c,Td,2d,Qs,None
4,3,0/0,0/0,0/0,3550,1,5,0,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4,2/25,0/0,0/0,782,2,10,0,8d,3d,Kc,None,None
96,4,3/60,2/80,2/120,2224,4,0,0,5d,2d,Td,3h,2c
97,4,3/60,2/80,2/120,690,2,70,0,5d,2d,Td,3h,2c
98,4,3/60,2/80,2/120,1165,1,20,0,5d,2d,Td,3h,2c


In [6]:
#Because pots include the number of players left in the hand, we split the remaining players from the cells
strp_flop_df = hands_df
strp_flop_df['flop_players'] = strp_flop_df['flop_pot'].str.split('/').str[0]
strp_flop_df['flop_pot'] = strp_flop_df['flop_pot'].str.split('/').str[1]
strp_flop_df['turn_players'] = strp_flop_df['turn_pot'].str.split('/').str[0]
strp_flop_df['turn_pot'] = strp_flop_df['turn_pot'].str.split('/').str[1]
strp_flop_df['river_players'] = strp_flop_df['river_pot'].str.split('/').str[0]
strp_flop_df['river_pot'] = strp_flop_df['river_pot'].str.split('/').str[1]
strp_flop_df.head()

,num_players,flop_pot,turn_pot,river_pot,chip_count,chair_num,player_bet,player_winnings,table_1,table_2,table_3,table_4,table_5,flop_players,turn_players,river_players
0,2,0,0,0,1995,1,10,20,None,None,None,None,None,0,0,0
1,2,0,0,0,50,2,10,0,None,None,None,None,None,0,0,0
2,2,20,20,0,1045,2,970,1920,8c,Td,2d,Qs,None,2,2,0
3,2,20,20,0,1000,1,950,0,8c,Td,2d,Qs,None,2,2,0
4,3,0,0,0,3550,1,5,0,None,None,None,None,None,0,0,0


In [7]:
#make sure numerical columns have numerical data type
strp_flop_df.dtypes

num_players         int64
flop_pot           object
turn_pot           object
river_pot          object
chip_count          int64
chair_num           int64
player_bet          int64
player_winnings     int64
table_1            object
table_2            object
table_3            object
table_4            object
table_5            object
flop_players       object
turn_players       object
river_players      object
dtype: object

In [10]:
#correct the numerical columns
strp_flop_df = strp_flop_df.astype({'flop_pot': 'int64',
                    'turn_pot': 'int64',
                    'river_pot': 'int64',
                    'flop_players': 'int64',
                    'turn_players': 'int64',
                    'river_players': 'int64'})
strp_flop_df.dtypes

num_players         int64
flop_pot            int64
turn_pot            int64
river_pot           int64
chip_count          int64
chair_num           int64
player_bet          int64
player_winnings     int64
table_1            object
table_2            object
table_3            object
table_4            object
table_5            object
flop_players        int64
turn_players        int64
river_players       int64
dtype: object

In [11]:
#create dummies for table cards to allow selection of cards on table
learning_df = pd.get_dummies(strp_flop_df)
learning_df.dtypes

num_players    int64
flop_pot       int64
turn_pot       int64
river_pot      int64
chip_count     int64
               ...  
table_5_Qs     uint8
table_5_Tc     uint8
table_5_Td     uint8
table_5_Th     uint8
table_5_Ts     uint8
Length: 271, dtype: object

In [12]:
#machine learning to estimate player winnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

In [13]:
#create targets by using a winning pot to determine success
X = learning_df.drop(['player_winnings'], axis=1)
y = learning_df['player_winnings']
new_y = []
for y_t in y:
    if y_t > 0:
        y_t = 1
    else:
        y_t = 0
    new_y.append(y_t)
y = np.array(new_y)

In [14]:
X.head()

,num_players,flop_pot,turn_pot,river_pot,chip_count,chair_num,player_bet,flop_players,turn_players,river_players,...,table_5_Kh,table_5_Ks,table_5_Qc,table_5_Qd,table_5_Qh,table_5_Qs,table_5_Tc,table_5_Td,table_5_Th,table_5_Ts
0,2,0,0,0,1995,1,10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,50,2,10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,20,20,0,1045,2,970,2,2,0,...,0,0,0,0,0,0,0,0,0,0
3,2,20,20,0,1000,1,950,2,2,0,...,0,0,0,0,0,0,0,0,0,0
4,3,0,0,0,3550,1,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [16]:
scaler = StandardScaler().fit(X_train)

In [17]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=270))
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
3275/3275 [==============================] - 3s 868us/step - loss: 0.3542 - accuracy: 0.8230
Epoch 2/50
3275/3275 [==============================] - 3s 858us/step - loss: 0.2633 - accuracy: 0.8641
Epoch 3/50
3275/3275 [==============================] - 3s 859us/step - loss: 0.2390 - accuracy: 0.8736
Epoch 4/50
3275/3275 [==============================] - 3s 866us/step - loss: 0.2309 - accuracy: 0.8762
Epoch 5/50
3275/3275 [==============================] - 3s 866us/step - loss: 0.2266 - accuracy: 0.8783
Epoch 6/50
3275/3275 [==============================] - 3s 860us/step - loss: 0.2248 - accuracy: 0.8801
Epoch 7/50
3275/3275 [==============================] - 3s 853us/step - loss: 0.2226 - accuracy: 0.8813
Epoch 8/50
3275/3275 [==============================] - 3s 856us/step - loss: 0.2213 - accuracy: 0.8820
Epoch 9/50
3275/3275 [==============================] - 3s 858us/step - loss: 0.2205 - accuracy: 0.8819
Epoch 10/50
3275/3275 [==============================] - 3s 861u

In [19]:
type(nn_model)

keras.engine.sequential.Sequential

In [20]:
len(y_train)

104788

In [19]:
#create and use the model with new data
#Inputs are: 
#number of players, 
#pot at the flop, 
#pot at the turn, 
#pot at the river, 
#player chip count, 
#player chair num, 
#player bet amount,
#players at the flop,
#players at the turn,
#players at the river,
#cards on the table

In [21]:
#get inputs
x_predict = [2,0,0,0,1995,1,10,0,0,0,'None','None','None','None','None']

In [22]:
#create dataframe with dummies columns
#start withe list of valid cards
cards = ['Ad','2d','3d','4d','5d','6d','7d','8d','9d','Td','Jd','Qd','Kd','Ah','2h','3h','4h','5h','6h','7h','8h','9h','Th','Jh','Qh','Kh','As','2s','3s','4s','5s','6s','7s','8s','9s','Ts','Js','Qs','Ks','Ac','2c','3c','4c','5c','6c','7c','8c','9c','Tc','Jc','Qc','Kc']

#create dummies reference list
dummies_cards = x_predict[:10]

#get the headers from the dummies columns
dummies_columns = list(X.columns.values)
dummies_columns = dummies_columns[10:]

#loop through each table card and add the corresponding binary value
i = 0
k = 0
while i < 5:
    i = i+1
    #get the current card
    curr_card = f'table_{i}_{x_predict[9+i]}'
    print(f"Adding card {i} : {curr_card}")
    j = 0
    while j<52:
        check_card = dummies_columns[k]
        if curr_card == check_card:
            print(f"Added {check_card}")
            dummies_cards.append(1)
        else:
            dummies_cards.append(0)
        j = j+1
        k = k+1
print(len(dummies_cards))

Adding card 1 : table_1_None
Adding card 2 : table_2_None
Adding card 3 : table_3_None
Adding card 4 : table_4_None
Adding card 5 : table_5_None
270


In [23]:
x_predict = np.array([dummies_cards])
x_predict.flatten()
x_predict.shape

(1, 270)

In [24]:
y_predict = nn_model.predict(x_predict)

1/1 [==============================] - 0s 69ms/step


In [30]:
result = nn_model.evaluate(x_predict,y_predict,verbose=1)
print(type(result))
result

1/1 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000
<class 'list'>


[0.0, 1.0]

In [26]:
y_predict

array([[0.]], dtype=float32)

In [27]:
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 2168      
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 2,249
Trainable params: 2,249
Non-trainable params: 0
_________________________________________________________________


In [28]:
nn_model.save('poker_model.hdf5')

In [31]:
dummies_columns = list(X.columns.values)
dummies_columns = dummies_columns[10:]

In [33]:
#file = open("dummies_columns.txt", "w")
with open("file.txt", "w") as output:
    output.write(str(dummies_columns))